# Caso Práctico - Visualización de Datos
---
Visualización de Datos del Máster Universitario en Ingeniería y Ciencia de Datos (2023/2024)

Alumno: **Javier Orive Soto**


# 1. Importación de dependencias

In [61]:
import pandas as pd
import panel as pn
pn.extension('vega')

import numpy as np
import altair as alt

import datetime as dt

# 2. Importación de datos

## 2.1 Datos de generación

### 2.1.1. Grafico de generacion por tecnología (días)

In [62]:
# Cargar el archivo Excel
#file_path = 'Datos/Economico/PrecioMedioHorarioFinal_2023_d.xls'
file_path = 'Datos/Generacion/GeneracionTotal_2023_h.csv'
#df = pd.read_excel(file_path)
df_G = pd.read_csv(file_path, delimiter=';')

# Seleccionar solo las columnas de interés
#df = df[['name', 'value', 'datetime']]

In [63]:
# Convertir la columna datetime a tipo datetime
df_G['datetime'] = pd.to_datetime(df_G['datetime'], utc = True) #, utc = True

# Convertir la columna value a tipo float
df_G['value'] = df_G['value'].astype(float)

# Convertir la columna name a tipo categoría
df_G['name'] = df_G['name'].astype('category')

# Elimino las columnas vacías
df_G = df_G.drop(['geoid', 'geoname'], axis=1)

df_G = df_G.drop(df_G[df_G['id']== 10195].index)
df_G['name'] = df_G['name'].str.slice(18)
df_G['date'] = df_G['datetime'].dt.date

df_G['year'] = df_G['datetime'].dt.year
df_G['month'] = df_G['datetime'].dt.month
df_G['day'] = df_G['datetime'].dt.day
df_G['hour'] = df_G['datetime'].dt.hour

In [65]:
df_G.head()

,id,name,value,datetime,date
4321,1169,Biogás,83.937,2022-12-31 23:00:00+00:00,2022-12-31
4322,1169,Biogás,81.609,2023-01-01 00:00:00+00:00,2023-01-01
4323,1169,Biogás,80.939,2023-01-01 01:00:00+00:00,2023-01-01
4324,1169,Biogás,80.873,2023-01-01 02:00:00+00:00,2023-01-01
4325,1169,Biogás,80.765,2023-01-01 03:00:00+00:00,2023-01-01


In [66]:
df_G.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155473 entries, 4321 to 164210
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   id        155473 non-null  int64              
 1   name      155473 non-null  object             
 2   value     155473 non-null  float64            
 3   datetime  155473 non-null  datetime64[ns, UTC]
 4   date      155473 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(2)
memory usage: 7.1+ MB


In [67]:
# https://towardsdatascience.com/how-to-build-a-time-series-dashboard-in-python-with-panel-altair-and-a-jupyter-notebook-c0ed40f02289
date_slider = pn.widgets.DateSlider(name='Date Slider', start=dt.datetime(2023, 1, 1), end=dt.datetime(2023, 12, 31), value=dt.datetime(2023, 1, 1))

date_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'5414a2f3-d617-4c9e-a9d2-8c3e1ec8273a': {'version…

In [68]:
@pn.depends(date_slider.param.value)
def get_plot(date):
     # Load and format the data
     df = df_G # define df
     ##df[‘date’] = pd.to_datetime(df[‘date’])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_slider.value
     # create filter mask for the dataframe
     mask = (df['date'] == start_date)
     df = df.loc[mask] # filter the dataframe

     selection = alt.selection_point(fields=['name'], bind='legend')

     # create the Altair chart object
     chart = alt.Chart(df).mark_area(interpolate='step').encode(
               x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
               y=alt.Y('sum(value):Q', title='Potencia eléctrica (MW)', scale=alt.Scale(domain=[0, 42000])),  # Suma de potencia
               color=alt.Color('name:N', title='Tecnología'),  # Diferenciar por tecnología
               opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
               ).properties(
               title='Generación eléctrica por tecnología',
               width=700,
               height=400
               ).add_params(selection)

     return chart

In [69]:
title = '## Desglose de generación eléctrica'
subtitle = 'Este dashboard permite al usuario seleccionar\n el día para el que se quieren visualizar los datos.\n\n Además, también se puede seleccionar la tecnología\n para resaltarla en el gráfico.'

dashboard = pn.Row(get_plot,
                   pn.Column(title, subtitle, date_slider)
)

In [70]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'fdf9fb3c-b785-483c-8daa-48501130fbe3': {'version…

### 2.1.2. Grafico de media anual generacion por tecnología

In [82]:
df_aux = df_G.groupby('name')['value'].sum().sort_values(ascending=False).reset_index()

# Calcular la suma total de la columna 'value'
total_sum = df_aux['value'].sum()

# Normalizar la columna 'value' a un 100%
df_aux['value_normalized'] = (df_aux['value'] / total_sum)

In [83]:
df_aux

,name,value,value_normalized
0,Eólica terrestre,6.108076e+07,2.419517e-01
1,Nuclear,5.412415e+07,2.143953e-01
2,Ciclo combinado,3.920130e+07,1.552832e-01
3,Solar fotovoltaica,3.648300e+07,1.445156e-01
4,Hidráulica UGH,2.068712e+07,8.194533e-02
5,Gas Natural Cogeneración,1.653210e+07,6.548655e-02
6,Turbinación bombeo,5.184742e+06,2.053767e-02
7,Solar térmica,4.673006e+06,1.851060e-02
8,Hidráulica no UGH,4.547275e+06,1.801255e-02
9,Hulla antracita,2.933010e+06,1.161817e-02


In [88]:
selection = alt.selection_point(fields=['name'], bind='legend')

# Base del gráfico
base = alt.Chart(df_aux).transform_window(
    rank='rank(value)',
    sort=[alt.SortField('value', order='descending')]
).encode(
    alt.Theta("value:Q", sort=df_aux['name'].to_list()).stack(True),
    alt.Radius("value").scale(type="sqrt", zero=True),
    color=alt.Color("name:N", title='Tecnología'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Generación eléctrica anual por tecnología',
    width=400,
    height=400
)

# Gráfico de arco
c1 = base.mark_arc(innerRadius=20, stroke="#fff")

# Gráfico de texto con formato
c2 = base.mark_text(radiusOffset=10).encode(
    text=alt.Text("value_normalized:Q", format=".0%")
    ).transform_filter(
    alt.datum.rank <= 6
)

# Mostrar el gráfico
c1 + c2

alt.LayerChart(...)

In [76]:
selection = alt.selection_point(fields=['name'], bind='legend')

base = alt.Chart(df_aux).encode(
    alt.Theta("value:Q", sort=df_aux['name'].to_list()).stack(True),
    alt.Radius("value").scale(type="sqrt", zero=True),
    color=alt.Color("name:N", title='Tecnología'),
    #color="name:N",
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection
).properties(
title='Generación eléctrica por tecnología',
width=400,
height=300)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="value_normalized")

c1 + c2

alt.LayerChart(...)

## Streamlit

In [ ]:
import streamlit as st

In [ ]:
df = df.pivot(index='datetime', columns='name', values='value').reset_index()

In [ ]:
df

In [ ]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='Precio medio horario componente banda secundaria:Q'
)

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='value:Q',
  color='name:N'
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N'
)

In [ ]:
selection = alt.selection_point(fields=['name'], bind='legend')

In [ ]:
# Plotting
chart.mark_line().encode(
    x='datetime:T',
    y='value:Q',
    tooltip=['datetime:T', 'value:Q']
).properties(
    title='Hourly Average Price Over Time',
    width=800,
    height=400
).interactive()





In [ ]:
chart.show

In [ ]:
# Crear el gráfico apilado
chart.mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()


In [ ]:
# Crear el gráfico apilado
alt.Chart(df).mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()